In [1]:
from PIL import Image
from imagehash import phash, dhash, average_hash
import os
import random
import shelve
from sys import getsizeof
from copy import deepcopy

In [2]:
def load_image_set(path):
    return load_image_sample(path, sample=1.0)

def load_image_sample(path, sample): 
    _docs = {doc: os.path.join(path, doc) for doc in os.listdir(path) if doc.endswith('.jpg')}
    return {doc: _docs[doc] for doc in random.sample(_docs.keys(), int(len(_docs)*sample))}

def fingerprint_images(images):
    return {img: str(dhash(Image.open(images[img]))) for img in images}

In [3]:
query_set = load_image_set('/Users/zubin.john/forge/image-dedup/Transformed_dataset/Query/')
test_set = load_image_set('/Users/zubin.john/forge/image-dedup/Transformed_dataset/Retrieval/')

In [4]:
test_hashes = fingerprint_images(test_set)
query_hashes = fingerprint_images(query_set)

In [7]:
# Create a reverse-lookup dictionary (inverted index) to map fingerprints to images

image_hashes = deepcopy(test_hashes)
image_hashes.update(query_hashes)
getsizeof(image_hashes)

hashed_images = {image_hashes[image]: image for image in image_hashes}

In [8]:
# Instantiate a custom-object database
db = shelve.open('../Transformed_dataset/imageset', writeback = True)

# Add all images in our dataset to the document index 
for each in image_hashes:
    db[image_hashes[each]] = db.get(image_hashes[each], []) + [each]

# Close the shelf database
db.close()

In [15]:
db = shelve.open('../Transformed_dataset/imageset')
R = {hashed_images[query]: db[query][:5] for query in query_hashes.values()}
db.close()